In [1]:
#| include: false
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
cd(@__DIR__)

  Activating project at `~/gitrepos/kdheepak.github.io/blog/effect-of-type-inference-on-performance-in-julia`


In [58]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 10 × Apple M1 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 8 virtual cores)
Environment:
  JULIA_PROJECT = @.


In [2]:
abstract type Shape end
area(::Shape) = 0.0

struct Square <: Shape
    side::Float64
end
area(s::Square) = s.side * s.side
    
struct Rectangle <: Shape
    width::Float64
    height::Float64
end
area(r::Rectangle) = r.width * r.height
    
struct Triangle <: Shape
    base::Float64
    height::Float64
end
area(t::Triangle) = 1.0/2.0 * t.base * t.height

struct Circle <: Shape
    radius::Float64
end
area(c::Circle) = π * c.radius^2

area (generic function with 5 methods)

In [3]:
using Test
@testset "Areas" begin
    @test area(Square(2)) == 4
    @test area(Rectangle(2,3)) == 6
    @test area(Triangle(2,3)) == 3
    @test area(Circle(2)) ≈ 4π
end;

Test Summary: | Pass  Total  Time
Areas         |    4      4  0.2s


In [4]:
using Random

Random.seed!(42)

function shape_builder(choice::Integer)
    if choice == 1
        Square(rand())
    elseif choice == 2
        Rectangle(rand(), rand())
    elseif choice == 3
        Triangle(rand(), rand())
    elseif choice == 4
        Circle(rand())
    end
end

count = 1_000_000
shapes = [shape_builder(rand((1,2,3,4))) for _ in 1:count];

In [5]:
#| echo: false
using Format
using Markdown
l = cfmt("%'d", length(shapes))
Markdown.md"The total number of shapes we have is $l."

The total number of shapes we have is 1,000,000.


In [6]:
typeof(shapes)

Vector{Shape} (alias for Array{Shape, 1})

In [101]:
main1(shapes) = sum(area, shapes)

main1 (generic function with 1 method)

In [102]:
@time main1(shapes);

  0.078527 seconds (2.00 M allocations: 30.552 MiB, 2.96% compilation time)


In [103]:
using BenchmarkTools

In [104]:
@benchmark main1(shapes)

BenchmarkTools.Trial: 80 samples with 1 evaluation.
 Range (min … max):  60.275 ms … 68.606 ms  ┊ GC (min … max): 0.00% … 8.34%
 Time  (median):     61.301 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   63.028 ms ±  2.966 ms  ┊ GC (mean ± σ):  2.59% ± 3.72%

     ▅▅ ▄█                                                     
  ▅█▆█████▃▅▁▅▆▁▁▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▅▆▆██▃▅▁▁▁▅ ▁
  60.3 ms         Histogram: frequency by time        68.4 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 1999999.

In [105]:
shapes_by_type(::Type{T}, shapes) where T = [shape for shape in shapes if isa(shape, T)]

square_arr = shapes_by_type(Square, shapes)
rectangle_arr = shapes_by_type(Rectangle, shapes)
triangle_arr = shapes_by_type(Triangle, shapes)
circle_arr = shapes_by_type(Circle, shapes)

nothing

In [106]:
typeof(square_arr)

Vector{Square} (alias for Array{Square, 1})

In [107]:
typeof(rectangle_arr)

Vector{Rectangle} (alias for Array{Rectangle, 1})

In [108]:
typeof(triangle_arr)

Vector{Triangle} (alias for Array{Triangle, 1})

In [109]:
typeof(circle_arr)

Vector{Circle} (alias for Array{Circle, 1})

In [110]:
filter_shapes_by_type(::Type{T}, shapes) where T = filter(s -> isa(s, T), shapes)

shape_arr1 = filter_shapes_by_type(Square, shapes)
shape_arr2 = filter_shapes_by_type(Rectangle, shapes)
shape_arr3 = filter_shapes_by_type(Triangle, shapes)
shape_arr4 = filter_shapes_by_type(Circle, shapes)

nothing

In [111]:
typeof(shape_arr1)

Vector{Shape} (alias for Array{Shape, 1})

In [112]:
typeof(shape_arr2)

Vector{Shape} (alias for Array{Shape, 1})

In [113]:
typeof(shape_arr3)

Vector{Shape} (alias for Array{Shape, 1})

In [114]:
typeof(shape_arr4)

Vector{Shape} (alias for Array{Shape, 1})

In [115]:
sorted_shapes_shape = vcat(square_arr, rectangle_arr, triangle_arr, circle_arr);
sorted_shapes_any = Any[s for s in sorted_shapes_shape];

In [116]:
typeof(sorted_shapes_shape)

Vector{Shape} (alias for Array{Shape, 1})

In [117]:
@benchmark main1(sorted_shapes_shape)

BenchmarkTools.Trial: 91 samples with 1 evaluation.
 Range (min … max):  52.558 ms … 61.063 ms  ┊ GC (min … max): 0.00% … 8.98%
 Time  (median):     53.446 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   55.196 ms ±  2.960 ms  ┊ GC (mean ± σ):  3.14% ± 4.39%

   ▁█  ▁                                                       
  ▅█████▇▆▃▆▃▄▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▃▇▅▃▇▅▃▄▃▁▁▁▃▃ ▁
  52.6 ms         Histogram: frequency by time        60.9 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 1999999.

In [118]:
typeof(sorted_shapes_any)

Vector{Any} (alias for Array{Any, 1})

In [119]:
@benchmark main1(sorted_shapes_any)

BenchmarkTools.Trial: 90 samples with 1 evaluation.
 Range (min … max):  53.054 ms … 61.621 ms  ┊ GC (min … max): 0.00% … 9.07%
 Time  (median):     54.299 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   55.898 ms ±  3.042 ms  ┊ GC (mean ± σ):  3.07% ± 4.37%

  ▁▄██▄▁▃   ▁                                            ▆  ▁  
  ███████▇▄▄█▆▇▆▄▁▄▆▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▄▄▄▆▆▄▄▄▆▁█▁▆█ ▁
  53.1 ms         Histogram: frequency by time        61.2 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 1999999.

In [120]:
main2(arrs...) = sum(main1, arrs)

main2 (generic function with 1 method)

In [121]:
@time main2(shape_arr1, shape_arr2, shape_arr3, shape_arr4);

  0.072885 seconds (2.01 M allocations: 31.174 MiB, 8.57% gc time, 15.24% compilation time: 79% of which was recompilation)


In [122]:
@benchmark main2(shape_arr1, shape_arr2, shape_arr3, shape_arr4)

BenchmarkTools.Trial: 91 samples with 1 evaluation.
 Range (min … max):  52.684 ms … 60.947 ms  ┊ GC (min … max): 0.00% … 9.20%
 Time  (median):     53.460 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   55.288 ms ±  2.943 ms  ┊ GC (mean ± σ):  3.14% ± 4.39%

    ▃▇▇█                                                   ▃   
  ▅█████▆▇▅▆▃▅▁▅▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▅▅▃▅▇▆█▇ ▁
  52.7 ms         Histogram: frequency by time          60 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 2000005.

In [123]:
@code_llvm main2(shape_arr1, shape_arr2, shape_arr3, shape_arr4)

;  @ In[120]:1 within `main2`
define nonnull {}* @japi1_main2_3639({}* %function, {}** noalias nocapture noundef readonly %args, i32 %nargs) #0 {
top:
  %0 = alloca [6 x {}*], align 8
  %stackargs = alloca {}**, align 8
  %.sub = getelementptr inbounds [6 x {}*], [6 x {}*]* %0, i64 0, i64 0
  store volatile {}** %args, {}*** %stackargs, align 8
; ┌ @ reduce.jl:535 within `sum`
; │┌ @ reduce.jl:535 within `#sum#305`
; ││┌ @ reduce.jl:307 within `mapreduce`
; │││┌ @ reduce.jl:307 within `#mapreduce#302`
; ││││┌ @ reduce.jl:175 within `mapfoldl`
; │││││┌ @ reduce.jl:175 within `#mapfoldl#298`
; ││││││┌ @ reduce.jl:44 within `mapfoldl_impl`
; │││││││┌ @ reduce.jl:48 within `foldl_impl`
; ││││││││┌ @ reduce.jl:68 within `_foldl_impl`
           %boundscheck.not = icmp eq i32 %nargs, 0
           br i1 %boundscheck.not, label %fail, label %pass

L8:                                               ; preds = %pass11
; ││││││││└
; ││││││││ @ reduce.jl:49 within `foldl_impl`
; ││││││││┌ @ reduce.j

In [124]:
@code_warntype main2(shape_arr1, shape_arr2, shape_arr3, shape_arr4)

MethodInstance for main2(::Vector{Shape}, ::Vector{Shape}, ::Vector{Shape}, ::Vector{Shape})
  from main2(arrs...) @ Main In[120]:1
Arguments
  #self#::Core.Const(main2)
  arrs::NTuple{4, Vector{Shape}}
Body::Any
1 ─ %1 = Main.sum(Main.main1, arrs)::Any
└──      return %1



In [126]:
@time main2(square_arr, rectangle_arr, triangle_arr, circle_arr);

  0.006807 seconds (13.87 k allocations: 931.625 KiB, 94.34% compilation time: 67% of which was recompilation)


In [127]:
@benchmark main2(square_arr, rectangle_arr, triangle_arr, circle_arr)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  274.709 μs … 388.708 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     280.250 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   281.906 μs ±   5.157 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▃▇█▇▆▃▄▅▄▃▁▁▁                                         ▂
  ▂▂▃▄▅▄▇▇█████████████████▇█▇██▇█▆▇▇▇█▇▆█▇▇▇▇▇▇▆▇▅▇▅▆▅▅▅▅▄▆▄▅▅ █
  275 μs        Histogram: log(frequency) by time        305 μs <

 Memory estimate: 160 bytes, allocs estimate: 4.

In [128]:
@code_llvm main2(square_arr, rectangle_arr, triangle_arr, circle_arr)

;  @ In[120]:1 within `main2`
define double @julia_main2_3651({}* noundef nonnull align 16 dereferenceable(40) %0, {}* noundef nonnull align 16 dereferenceable(40) %1, {}* noundef nonnull align 16 dereferenceable(40) %2, {}* noundef nonnull align 16 dereferenceable(40) %3) #0 {
top:
; ┌ @ reduce.jl:535 within `sum`
; │┌ @ reduce.jl:535 within `#sum#305`
; ││┌ @ reduce.jl:307 within `mapreduce`
; │││┌ @ reduce.jl:307 within `#mapreduce#302`
; ││││┌ @ reduce.jl:175 within `mapfoldl`
; │││││┌ @ reduce.jl:175 within `#mapfoldl#298`
; ││││││┌ @ reduce.jl:44 within `mapfoldl_impl`
; │││││││┌ @ reduce.jl:48 within `foldl_impl`
; ││││││││┌ @ reduce.jl:68 within `_foldl_impl`
; │││││││││┌ @ operators.jl:544 within `afoldl`
; ││││││││││┌ @ reduce.jl:100 within `MappingRF`
; │││││││││││┌ @ In[101]:1 within `main1`
; ││││││││││││┌ @ reducedim.jl:1011 within `sum`
; │││││││││││││┌ @ reducedim.jl:1011 within `#sum#829`
; ││││││││││││││┌ @ reducedim.jl:1015 within `_sum`
; │││││││││││││││┌ @ reducedi

In [129]:
@code_warntype main2(square_arr, rectangle_arr, triangle_arr, circle_arr)

MethodInstance for main2(::Vector{Square}, ::Vector{Rectangle}, ::Vector{Triangle}, ::Vector{Circle})
  from main2(arrs...) @ Main In[120]:1
Arguments
  #self#::Core.Const(main2)
  arrs::Tuple{Vector{Square}, Vector{Rectangle}, Vector{Triangle}, Vector{Circle}}
Body::Float64
1 ─ %1 = Main.sum(Main.main1, arrs)::Float64
└──      return %1

